# TP 2:  Computational Graph



During last TP we asked you to make the backward pass, implementing all the derivatives needed. 
As you can expect, doing this every time you make a new model is a little redundant. 
The ML libraries allow you to implement models by focusing only on the forward pass, they construct than a graph and compute derivatives from the bottom to the leafs of the graph. This graph is known as "computational graph".

The aim of this TP is to build a computational graph inspired by [pytorch](https://pytorch.org/), and than test it with a simple model (MLP).
The construction of the model, of the loss and the optimizer are also inspired by pytorch.

The transition to pytorch should be easy in the future.

**Disclaimer** This code is inspired by how users use Pytorch, it doesn't replace Pytorch and the implemtatin differ form Pytorch. The only goal of this TP is to give an intuition how pytorch and the computational graph work, before starting using it!

## The Computational Graph

The computational graph is a graph that specifies the operations done to get a given value. 
If C = A + B, the graph will look like:
```
A
 \
  \
   + -- C 
  /
 /
B
```
or if F = C * D and E = log(F), the graph will look like:
```
A
 \
  \
   + -- C
  /      \
 /        \
B          * -- F -- log -- E
          /
         /
        D
```

As you can see, the graph is build during the "forward pass" and it is easy to see the gradients flow (start on E and end on the leafs A, B and D. 

### Variable
To build this graph in our code, we introduce an object called Variable. This object will look like an array in numpy, it will content data, and have methods like mean, sum, t, etc. Variable has also a grad, a grad__fn, and children field.

* Variable.grad: store the gradients during the backward pass for the given variable (same shape as Variabale.shape)
* Variable.grad_fn: store the function that has built this Variable (addition, multiplication, etc)
* Variable.children: list of all the operations where the Variable was used. We need Variable.children during backward pass to know if all the children have propagate thier gradients before the current variable compute in turn its gradients.

### Functions

Functions cointains all the oprations we can use in your code. Each operation needs a forward and a backward method.
The forward method is simply the __init__ method where you compute and store the result of the operation. 
There is 2 backward methods:
* backward (general): Inherited from the _Function parent class, it calls the second _backward method (see below) and updates the gradients of the variables used to build the current one.
* _backward (specific): Is specific to each operation, computes the gradients for its parents. The derivatives are computed according to the specific operation.

### Functional

Is simply an interface for all functions defined in functions.
You don't need to touch this file. but take a look at it, because it gives you all the functions you have add.
If you don't use a function from this interface, you will not be able to construct the graph and ptopagate trough it.

*|!\* Even the standart operation you can use directly: +, -, *, / use the operation of functional! Take a look at Variable.__add__, Variable.__sub__, Variable.__mul__, Variable.__truediv__ if you doubt.


## What we ask you to do:

Complete the *Fill here* in the following cells of this notebook.
Use latex notation to add your formulas. 
Once you have filled the missing parts of an operation, go to function.py and implement the missing parts of it. Tests are provided by saving the gradients using pytorch on the same conditions (same arrays, gradients cleared between operations).


Before starting with the derivatives, let's take a look at variable.py.
The majority of this class is provided to you. We **ask you to describe a little bit this class in your report, 
mostly the methods backward and update_grad**. Than fill in the missing parts in this two methods.

We will now create the Variables you will use to test your implementation.

In [1]:
# for auto-reloading external modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

from functional import F
from variable import Variable

def display_variable_information(name, var):
    print("\nName:", name)
    print("Data:\n", var.data)
    print("Shape:", var.shape)
    print("Grad:\n", var.grad)
    print("Grad_fn:", var.grad_fn)

# scalars
a = Variable([4.5])
b = Variable([6.78])

# arrays
C = Variable([[1.73, 2.83], [5.13, 8.43], [5.13, 8.43]])
D = Variable([[3.57, 4.96], [2.06, 1.94], [5.13, 8.43]])


print("Variables Informations:")
# uncomment if you want
display_variable_information("a", a)
#display_variable_information("b", b)
#display_variable_information("c", C)
#display_variable_information("d", D)

Variables Informations:

Name: a
Data:
 [[4.5]]
Shape: (1, 1)
Grad:
 None
Grad_fn: None


In [15]:
print(C)
print(C[0,1])

C[0,0] = 10

print(C)

m = C + D
m = F.add(C, D)


k = C.sum()
print(k.grad_fn)
print(k)

Variable([[10.    2.83],
          [ 5.13  8.43],
          [ 5.13  8.43]])
Variable([[2.83]])
Variable([[10.    2.83],
          [ 5.13  8.43],
          [ 5.13  8.43]])
None
Variable([[39.95]])


In [ ]:
from check_values import check_result_and_grads

def clear_variables(*argv):
    """Clear all Variables passed in arguments."""
    for var in argv:
        var.grad = None
        var.grad_fn = None
        var.children = []
        var.retained_values = {}
        
        

# Addition

**Given to you as example** 

**Inputs**: $x, y$

**Operation**: $f(x,y) = x + y$

**Derivatives**:
+ **w.r.t.** $x$: 
    
    $\frac{\partial f}{\partial x} = 1$
    
    By chain rule:
    $\frac{\partial}{\partial x} = \frac{\partial}{\partial f} \cdot \frac{\partial f}{\partial x} = \frac{\partial}{\partial f} \cdot 1 = \frac{\partial}{\partial f}$

+ **w.r.t.** $y$:

    $\frac{\partial f}{\partial y} = 1$
    
    By chain rule:
    $\frac{\partial}{\partial y} = \frac{\partial}{\partial f} \cdot \frac{\partial f}{\partial y} = \frac{\partial}{\partial f} \cdot 1 = \frac{\partial}{\partial f}$

In [ ]:
clear_variables(a, b, C, D)

res_scalar = a + b
res_scalar.backward()
check_result_and_grads(res_scalar, a, b, operation="addition", itype="scalar")

res_array = C + D
res_array.mean().backward()
check_result_and_grads(res_array, C, C, operation="addition", itype="array")

# Subtraction

**Inputs**: $x, y$

**Operation**: $f(x,y) = x - y$

**Derivatives**:
+ **w.r.t.** $x$: 
    
    $\frac{\partial f}{\partial x} = ...$ *Fill here*
    
    By chain rule:
    $\frac{\partial}{\partial x} = \frac{\partial}{\partial f} \cdot \frac{\partial f}{\partial x} = ...$ *Fill here*

+ **w.r.t.** $y$:

    $\frac{\partial f}{\partial y} = ...$ *Fill here*
    
    By chain rule:
    $\frac{\partial}{\partial y} = \frac{\partial}{\partial f} \cdot \frac{\partial f}{\partial y} = ...$ *Fill here*

In [ ]:
clear_variables(a, b, C, D)

res_scalar = a - b
res_scalar.backward()
check_result_and_grads(res_scalar, a, b, operation="subtraction", itype="scalar")

res_array = C - D
res_array.mean().backward()
check_result_and_grads(res_array, C, D, operation="subtraction", itype="array")

# Multiplication

**Inputs**: $x, y$

**Operation**: $f(x,y) = x * y$

**Derivatives**:
+ **w.r.t.** $x$: 
    
    $\frac{\partial f}{\partial x} = ...$ *Fill here*
    
    By chain rule:
    $\frac{\partial}{\partial x} = \frac{\partial}{\partial f} \cdot \frac{\partial f}{\partial x} = ...$ *Fill here*

+ **w.r.t.** $y$:

    $\frac{\partial f}{\partial y} = ...$ *Fill here*
    
    By chain rule:
    $\frac{\partial}{\partial y} = \frac{\partial}{\partial f} \cdot \frac{\partial f}{\partial y} = ...$ *Fill here*

In [ ]:
clear_variables(a, b, C, D)

res_scalar = a * b
res_scalar.backward()
check_result_and_grads(res_scalar, a, b, operation="multiplication", itype="scalar")

res_array = C * D
res_array.mean().backward()
check_result_and_grads(res_array, C, D, operation="multiplication", itype="array")

# Division

**Inputs**: $x, y$

**Operation**: $f(x,y) = x / y$

**Derivatives**:
+ **w.r.t.** $x$: 
    
    $\frac{\partial f}{\partial x} = ...$ *Fill here*
    
    By chain rule:
    $\frac{\partial}{\partial x} = \frac{\partial}{\partial f} \cdot \frac{\partial f}{\partial x} = ...$ *Fill here*

+ **w.r.t.** $y$:

    $\frac{\partial f}{\partial y} = ...$ *Fill here*
    
    By chain rule:
    $\frac{\partial}{\partial y} = \frac{\partial}{\partial f} \cdot \frac{\partial f}{\partial y} = ...$ *Fill here*

In [ ]:
clear_variables(a, b, C, D)

res_scalar = a / b
res_scalar.backward()
check_result_and_grads(res_scalar, a, b, operation="division", itype="scalar")

res_array = C / D
res_array.mean().backward()
check_result_and_grads(res_array, C, D, operation="division", itype="array")

# Matrix Multiplication

**Inputs**: $x, y$

**Operation**: $f(x,y) = x.dot(y)$

**Derivatives**:
+ **w.r.t.** $x$: 
    
    $\frac{\partial f}{\partial x} = ...$ *Fill here*
    
    By chain rule:
    $\frac{\partial}{\partial x} = \frac{\partial}{\partial f} \cdot \frac{\partial f}{\partial x} = ...$ *Fill here*

+ **w.r.t.** $y$:

    $\frac{\partial f}{\partial y} = ...$ *Fill here*
    
    By chain rule:
    $\frac{\partial}{\partial y} = \frac{\partial}{\partial f} \cdot \frac{\partial f}{\partial y} = ...$ *Fill here*

In [ ]:
clear_variables(a, b, C, D)

res_array = F.matmul(C.t(), D)
res_array.mean().backward()

check_result_and_grads(res_array, C, D, operation="matMul", itype="array")

# Exponential

**Inputs**: $x$

**Operation**: $f(x) = e^x$

**Derivatives**:
+ **w.r.t.** $x$: 
    
    $\frac{\partial f}{\partial x} = ...$ *Fill here*
    
    By chain rule:
    $\frac{\partial}{\partial x} = \frac{\partial}{\partial f} \cdot \frac{\partial f}{\partial x} = ...$ *Fill here*

In [ ]:
clear_variables(a, b, C, D)

res_scalar = F.exp(a)
res_scalar.backward()
check_result_and_grads(res_scalar, a, operation="exp", itype="scalar")

res_array = F.exp(C)
res_array.mean().backward()
check_result_and_grads(res_array, C, operation="exp", itype="array")

# Natural Logarithm

**Inputs**: $x$

**Operation**: $f(x) = ln(x)$

**Derivatives**:
+ **w.r.t.** $x$: 
    
    $\frac{\partial f}{\partial x} = ...$ *Fill here*
    
    By chain rule:
    $\frac{\partial}{\partial x} = \frac{\partial}{\partial f} \cdot \frac{\partial f}{\partial x} = ...$ *Fill here*

In [ ]:
clear_variables(a, b, C, D)

res_scalar = F.log(a)
res_scalar.backward()
check_result_and_grads(res_scalar, a, operation="log", itype="scalar")

res_array = F.log(C)
res_array.mean().backward()
check_result_and_grads(res_array, C, operation="log", itype="array")

# Sinus

**Inputs**: $x$

**Operation**: $f(x) = \sin(x)$

**Derivatives**:
+ **w.r.t.** $x$: 
    
    $\frac{\partial f}{\partial x} = ...$ *Fill here*
    
    By chain rule:
    $\frac{\partial}{\partial x} = \frac{\partial}{\partial f} \cdot \frac{\partial f}{\partial x} = ...$ *Fill here*

In [ ]:
clear_variables(a, b, C, D)

res_scalar = F.sin(a)
res_scalar.backward()
check_result_and_grads(res_scalar, a, operation="sin", itype="scalar")

res_array = F.sin(C)
res_array.mean().backward()
check_result_and_grads(res_array, C, operation="sin", itype="array")

# Cosinus

**Inputs**: $x$

**Operation**: $f(x) = \cos(x)$

**Derivatives**:
+ **w.r.t.** $x$: 
    
    $\frac{\partial f}{\partial x} = ...$ *Fill here*
    
    By chain rule:
    $\frac{\partial}{\partial x} = \frac{\partial}{\partial f} \cdot \frac{\partial f}{\partial x} = ...$ *Fill here*

In [ ]:
clear_variables(a, b, C, D)

res_scalar = F.cos(a)
res_scalar.backward()
check_result_and_grads(res_scalar, a, operation="cos", itype="scalar")

res_array = F.cos(C)
res_array.mean().backward()
check_result_and_grads(res_array, C, operation="cos", itype="array")

# Tangent

**Inputs**: $x$

**Operation**: $f(x) = \tan(x)$

**Derivatives**:
+ **w.r.t.** $x$: 
    
    $\frac{\partial f}{\partial x} = ...$ *Fill here*
    
    By chain rule:
    $\frac{\partial}{\partial x} = \frac{\partial}{\partial f} \cdot \frac{\partial f}{\partial x} = ...$ *Fill here*

In [ ]:
clear_variables(a, b, C, D)

res_scalar = F.tan(a)
res_scalar.backward()
check_result_and_grads(res_scalar, a, operation="tan", itype="scalar")

res_array = F.tan(C)
res_array.mean().backward()
check_result_and_grads(res_array, C, operation="tan", itype="array")

# Sigmoid

**Inputs**: $x$

**Operation**: $f(x) = \frac{1}{1 + e^{-x}}$

**Derivatives**:
+ **w.r.t.** $x$: 
    
    $\frac{\partial f}{\partial x} = ...$ *Fill here*
    
    By chain rule:
    $\frac{\partial}{\partial x} = \frac{\partial}{\partial f} \cdot \frac{\partial f}{\partial x} = ...$ *Fill here*

In [ ]:
clear_variables(a, b, C, D)

res_scalar = F.sigmoid(a)
res_scalar.backward()
check_result_and_grads(res_scalar, a, operation="sigmoid", itype="scalar")

res_array = F.sigmoid(C)
res_array[0,0].backward()
check_result_and_grads(res_array, C, operation="sigmoid", itype="array")

# Tanh

**Inputs**: $x$

**Operation**: $f(x) = \frac{e^{x} - e^{-x}}{e^{x} + e^{-x}}$

**Derivatives**:
+ **w.r.t.** $x$: 
    
    $\frac{\partial f}{\partial x} = ...$ *Fill here*
    
    By chain rule:
    $\frac{\partial}{\partial x} = \frac{\partial}{\partial f} \cdot \frac{\partial f}{\partial x} = ...$ *Fill here*

In [ ]:
clear_variables(a, b, C, D)

res_scalar = F.tanh(a)
res_scalar.backward()
check_result_and_grads(res_scalar, a, operation="tanh", itype="scalar")

res_array = F.tanh(C)
res_array[0,0].backward()
check_result_and_grads(res_array, C, operation="tanh", itype="array")

# ReLu

**Inputs**: $x$

**Operation**: $f(x) = \max(0, x)$

**Derivatives**:
+ **w.r.t.** $x$: 
    
    $\frac{\partial f}{\partial x} = ...$ *Fill here*
    
    By chain rule:
    $\frac{\partial}{\partial x} = \frac{\partial}{\partial f} \cdot \frac{\partial f}{\partial x} = ...$ *Fill here*

In [ ]:
clear_variables(a, b, C, D)

res_scalar = F.relu(a)
res_scalar.backward()
check_result_and_grads(res_scalar, a, operation="relu", itype="scalar")

res_array = F.relu(C)
res_array[0,0].backward()
check_result_and_grads(res_array, C, operation="relu", itype="array")

# Softmax

***The derivative of the softmax is not trivial to computein a vectorized manner, I have done the exercice and give you my implementation of the softmax, feel free to ask me questions about it.***

***You have to fill bellow the formulas!***

**Inputs**: $x$

**Operation**: $f(x) = \frac{e^{x_i}}{\sum_i e^{x_i}}$

**Derivatives**:
+ **w.r.t.** $x$: 
    
    $\frac{\partial f}{\partial x} = ...$ *Fill here*
    
    By chain rule:
    $\frac{\partial}{\partial x} = \frac{\partial}{\partial f} \cdot \frac{\partial f}{\partial x} = ...$ *Fill here*

In [ ]:
clear_variables(a, b, C, D)

res_array = F.softmax(C, dim=0)
res_array[0,0].backward()

check_result_and_grads(res_array, C, operation="softmax", itype="array")

# Cross Entropy Loss

For the Cross entropy loss we have used  the trick from pytorch that implements direclty the cross entropy loss with the softmax for more stability.

Take a look [here](https://pytorch.org/docs/stable/nn.html#crossentropyloss).

You don't have to implement it but make sure you understand what append here, comment it on your report.


In [ ]:
import nn as nn

X = Variable([[0.1711, 0.5140, 0.3149], [0.1359, 0.4985, 0.3656], [0.0275, 0.5467, 0.4258]])
y = Variable([1, 2, 0])

cel = nn.CrossEntropyLoss()

loss = cel(X, y)

loss.backward()

check_result_and_grads(res_array, X, operation="CEL", itype="array")

## An MLP as example

Now that you have all the components filled for the graph computational, you will need some additional steps to make a MLP trainable.

You have to complete the missing parts in ***nn.py*** and in ***optim.py***.


First we will generate a simple dataset, each color represents each class.
As you can see, we have 3 classes and each sample has 2 features (the cordinates).

In [ ]:
# for auto-reloading external modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

import numpy as np

import matplotlib
import matplotlib.pyplot as plt

import sklearn
import sklearn.datasets
import sklearn.linear_model

# Display plots inline and change default figure size
%matplotlib inline
matplotlib.rcParams['figure.figsize'] = (10.0, 8.0)

# Generate a dataset and plot it# Gener
N = 500
np.random.seed(0)
X, y = sklearn.datasets.make_blobs(N)
plt.scatter(X[:,0], X[:,1], s=40, c=y)

X_train = X[:350]
y_train = y[:350]
X_test = X[350:]
y_test = y[350:]

## Define and train the MLP

Follow the todos here and complete the missing parts in ***nn.py*** and ***optim.py***.

List of things you have to do. You can put a 'x' inside the [ ] when you have done it!
Example: * [x] Example done.



* nn.py:
    * Linear:
        * in init
            * [ ] Initialize the weights.
            * [ ] Initialize the bias.
        * in call:
            * [ ] Implement the linear transformation.
            * [ ] Add the bias.
* optim.py:
    * SGD:
        * in step:
            * [ ] Implement the SGD update mechanism.

In [ ]:
from functional import F
from variable import Variable
import nn as nn
from optim import SGD

np.random.seed(13)

class MLP(nn.Module):
    def __init__(self, in_features, hidden_size, out_features):
        #######################################################################
        # TODO: define 2 linear layers, one that takes the inputs and outputs
        # values with hidden_size
        # and the second one that takes the values from the first layer and
        # outputs the scores. 
        # implement Linear in nn.py before, you need it here.
        #######################################################################
        pass
        #######################################################################
        # --------------------------- END OF YOUR CODE ------------------------
        #######################################################################

        
    def forward(self, X):
        output = None
        #######################################################################
        # TODO: define your forward pass as follow
        #    1) y = linear(inputs)
        #    2) y_nl = relu(y)
        #    3) output = linear(y_nl)
        # softmax not needed because it's already in cross entropy
        #######################################################################
        pass
        #######################################################################
        # --------------------------- END OF YOUR CODE ------------------------
        #######################################################################
        return output


model = MLP(2, 100, 3)

optimizer = SGD(model.parameters(), lr=1e-3)
loss_fn = nn.CrossEntropyLoss()

epochs = 1000
batch_size = 50

history_losses = []
history_acc = []

for epoch in range(1, epochs+1):
    model.train()
    
    indices = range(X_train.shape[0])

    train_losses = []
    train_acc = []
    
    for iteration in range(X_train.shape[0]//batch_size):
        batch_indices = np.random.choice(indices, batch_size)
        indices = list(set(indices) - set(batch_indices))

        X_batch = Variable(X_train[batch_indices])
        y_batch = Variable(y_train[batch_indices])
        
        
        #######################################################################
        # TODO: Add here all the elements you need to train your model for each
        # batch.
        #######################################################################
        
        # you need to clear out the gradients for all the parameters
        pass
        
        # compute the forward pass
        pass
        
        # compute tht loss
        pass
        
        # compute the backward pass
        pass
        
        # optimize
        pass
        #######################################################################
        # --------------------------- END OF YOUR CODE ------------------------
        #######################################################################

        # keep loss
        train_losses.append(loss.item())
        
        # keep accuracy
        y_pred = np.argmax(outputs.data, axis=1)
        train_acc.append((y_pred[:, None] == y_batch.data).mean())
    
    history_losses.append(np.mean(train_losses))
    history_acc.append(np.mean(train_acc))
    
    # mod allow us to only display in a logaritmic way
    mod = 10**np.floor(np.log10(epoch))
    if epoch % mod == 0:
        print("Epoch {:>3}/{:>3}, loss {:.4f}, acc {:.2f}".format(epoch, epochs, history_losses[-1], history_acc[-1]))

## Visualisation

Now you can visualise for fun the loss and the accuracy of your model during trainning and get the final accuracy.

In [ ]:
plt.plot(history_losses, c="r", label="loss")
plt.legend()

In [ ]:
plt.plot(history_acc, c="g", label="Accuracy")
plt.legend()

You should get ~90% of accuracy with the test set.

In [ ]:
model.eval()
X_test_var = Variable(X_test)

outputs = model(X_test_var)

y_pred = np.argmax(outputs.data, axis=1)
acc = (y_pred == y_test).mean()

print("Accuracy on test set: {:.2f}".format(acc))